In [1]:

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from utils import iouUtils, calcIOU, lrScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
import sys, os
from keras.regularizers import l2
from YOLOv1_learning_Rate import customLearningRate
from YOLOv1_Reshape_Layer import YOLOv1_LastLayer_Reshape
from YOLOv1_Loss import YOLOv1_loss
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

here = os.path.dirname(".")
sys.path.append(os.path.join(here, '..'))

from dataHandler import *

2024-07-30 19:30:13.005110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 19:30:13.019000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 19:30:13.022676: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 19:30:13.032001: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-30 19:30:13.862765: W tensorflow/compiler/tf2

Num GPUs Available:  1


I0000 00:00:1722355214.794416   14212 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722355214.821316   14212 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722355214.821461   14212 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1722355214.822502   14212 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
import tensorflow as tf
from utils import iouUtils, calcIOU, customSQRT2


def iouUtils(boxParams, gridRatio = tf.constant(7, tf.float32)):
    """
    Given bounding box centers and its width and height, calculates top-left and bottom-right coordinates of the box.
    Note that calculations in this function are done with teh assumption of w and h being a float number, between 0 and 1
    with respect to the entire image's size. However, x and y of the bounding box's center are assumed to be a float 
    between 0 and 1, with respect to the upper-left point of the grid cell.

    Args:
        boxParams: tf.Tensor: A tensor with following information (Box center X, Box center Y, Box width, Box height) for all
            boxes in a tensor.
        gridRatio: int: The number of evenly distributed grid cells in each image axis. Use 7 for YOLOv1.
    
    Returns:
        Two tensors, one indicating top-left pint of the bBox and, the other one denoting bottom-right edge.
    """
    boxXY = boxParams[...,0:2]
    halfWH = tf.divide(boxParams[...,2:], tf.constant([2.]))

    # Top-left (X, Y) and bottom-right (X, Y)
    return tf.subtract(boxXY, halfWH * gridRatio), tf.add(boxXY, halfWH * gridRatio)

def calcIOU(predict_topLeft, predict_bottomRight, truth_topLeft, truth_bottomRight):
    """
    Calculates intersection over union for two bounding boxes.

    Args:
        predict_topLeft, predict_bottomRight: tf.Tensor: Top-left and bottom-right coordinates of the predicted box, acquired 
            by iouUtils.
        truth_topLeft, truth_bottomRight: tf.Tensor: Top-left and bottom-right coordinates of the ground truth box, acquired 
            by iouUtils.
    
    Returns:
        Intersection over union of two boxes
    """

    intersectEdgeLeft = tf.maximum(predict_topLeft, truth_topLeft)
    intersectEdgeRight = tf.minimum(predict_bottomRight, truth_bottomRight)
    
    intersectWH = tf.abs(tf.subtract(intersectEdgeLeft, intersectEdgeRight))
    intersectArea = tf.reduce_prod(intersectWH, axis = -1)

    # Get area of predicted and ground truth bounding boxes
    predArea = tf.reduce_prod(tf.abs(tf.subtract(predict_topLeft, predict_bottomRight)), axis = -1)
    truthArea = tf.reduce_prod(tf.abs(tf.subtract(truth_topLeft, truth_bottomRight)), axis = -1)

    
    # Return IOU
    return tf.divide(intersectArea, predArea + truthArea - intersectArea)

def lrScheduler(epoch, schedule, currentLR):
    """
    Returns a learning rate value with respect to epoch number.

    Args: 
        epoch: int: the current epoch number.
        schedule: list: A list of tuples of epoch number and its respective learning rate value. 
            If the epoch number of the fitting process doesn't reach the specified epoch number,
            the learning rate will remail unchanged. The entries have to be in order of epoch 
            numbers.
        currentLR: float: The learning rate of the model before starting the most recent epoch.

    Returns: float: learning rate.
    """
    
    newLR = currentLR

    for entry in schedule:
        if entry[0] == epoch:
            newLR = float(entry[1])
    
    return newLR



class customLearningRate(keras.callbacks.Callback):
    """
    Sets the learning rate of the fitting process with respect to the epoch number.

    Args:
        schedule: method: Using the epoch number, returns the suitable learning rate
        LR_schedule: list: A list of tuples of epoch number and its respective learning rate value. 
            If the epoch number of the fitting process doesn't reach the specified epoch number,
            the learning rate will remail unchanged. The entries have to be in order of epoch 
            numbers.
    """
    def __init__(self, scheduleFCN, LR_schedule):
        """
        Initialized the class

        Args: 
            scheduleFCN: method: A method that returns new learning rate
            LR_schedule: list: 
        """
        super(customLearningRate, self).__init__()
        self.LR_schedule = LR_schedule
        self.scheduleFCN = scheduleFCN

    def on_epoch_begin(self, epoch, logs=None):
        """
        Runs on the epoch start.

        Args:
            epoch: int: The current epoch number.
        """

        # # Check to see of the model has defined a learning rate
        # if hasattr(self.model.optimizer, "lr"):
        #     raise Exception("custom learning rate generator: First define a learning rate for the model.")
        
        # Get current learning rate
        learningRate = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))

        # Get the new learning rate
        newLearningRate = self.scheduleFCN(epoch, self.LR_schedule, learningRate)

        # Set the new learning rate as the model's learning rate
        
        self.model.optimizer.learning_rate.assign(newLearningRate)

        # Notify the user
        if learningRate != newLearningRate:
            tf.print(f"Updated the learning rate at epoch NO. {epoch}. New learning rate: {newLearningRate}")


class customLearningRate(keras.callbacks.Callback):
    """
    Sets the learning rate of the fitting process with respect to the epoch number.

    Args:
        schedule: method: Using the epoch number, returns the suitable learning rate
        LR_schedule: list: A list of tuples of epoch number and its respective learning rate value. 
            If the epoch number of the fitting process doesn't reach the specified epoch number,
            the learning rate will remail unchanged. The entries have to be in order of epoch 
            numbers.
    """
    def __init__(self, scheduleFCN, LR_schedule):
        """
        Initialized the class

        Args: 
            scheduleFCN: method: A method that returns new learning rate
            LR_schedule: list: 
        """
        super(customLearningRate, self).__init__()
        self.LR_schedule = LR_schedule
        self.scheduleFCN = scheduleFCN

    def on_epoch_begin(self, epoch, logs=None):
        """
        Runs on the epoch start.

        Args:
            epoch: int: The current epoch number.
        """

        # # Check to see of the model has defined a learning rate
        # if hasattr(self.model.optimizer, "lr"):
        #     raise Exception("custom learning rate generator: First define a learning rate for the model.")
        
        # Get current learning rate
        learningRate = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))

        # Get the new learning rate
        newLearningRate = self.scheduleFCN(epoch, self.LR_schedule, learningRate)

        # Set the new learning rate as the model's learning rate
        
        self.model.optimizer.learning_rate.assign(newLearningRate)

        # Notify the user
        if learningRate != newLearningRate:
            tf.print(f"Updated the learning rate at epoch NO. {epoch}. New learning rate: {newLearningRate}")


def YOLOv1_loss(yTrue, yPred):
    """
    Runs in the even of loss function calculations
    
    Args:
        yTrue, yPred: tf.Tensor: The ground truth value and the predicted value, respectively

    Returns:
        The calculated loss.
    """
    lambdaNoObj = tf.constant(.5)
    lambdaCoord = tf.constant(5.)

    # Split the predictions and ground truth vectors to coordinates, confidence and class matrices
    # 1. Ground truth 
    idx1, idx2 = 1, 1 + 1
    targetClass = yTrue[...,:idx1]
    targetConf = yTrue[...,idx1:idx2]
    targetCoords = yTrue[...,idx2:]

    # 2. Prediction
    idx1, idx2 = 1, 1 + 2
    predClass = yPred[...,:idx1]
    predConf = yPred[...,idx1:idx2]
    predCoords = yPred[...,idx2:]

    # Get the best bounding boxes by calculating the IOUs
    # Note: To to do this process for the confidence scores as well, we concat each box's confidence
    # score to its bounding box coordinates and analyze them as a whole.
    predBox1 = tf.concat([tf.expand_dims(predConf[...,0],-1),predCoords[...,:4]], axis = -1)
    predBox2 = tf.concat([tf.expand_dims(predConf[...,1],-1),predCoords[...,4:]], axis = -1)

    # Get the corners of bounding boxes to calculate IOUs
    # Note, iouUtils is not coded to accept confidence scores. So we only pass the coordinates into 
    # it. 
    p1_left, p1_right = iouUtils(predBox1[...,1:]) 
    p2_left, p2_right = iouUtils(predBox2[...,1:])
    t_left, t_right = iouUtils(targetCoords) 

    # Calculate IOUs for first and second predicted bounding box
    p1_IOU = calcIOU(p1_left, p1_right, t_left, t_right)
    p2_IOU = calcIOU(p2_left, p2_right, t_left, t_right)

    # Get the cells that have objects
    maskObj = tf.cast(0 < targetConf, tf.float32)
    maskNoObj = tf.cast(0 == targetConf, tf.float32)
    
    mask_p1Bigger = tf.expand_dims(tf.cast(p2_IOU < p1_IOU, tf.float32),-1)
    mask_p2Bigger = tf.expand_dims(tf.cast(p1_IOU <= p2_IOU, tf.float32),-1)

    # Getting the responsible bounding box for loss calculation. Output is of shape [...,5]
    # And the first element is the confidence score of that box.
    respBox = maskObj*(mask_p1Bigger * predBox1 + mask_p2Bigger * predBox2)

    # Calculating the losses
    # 1. Classification loss
    classificationLoss =  tf.math.reduce_sum(tf.math.square(maskObj * tf.subtract(targetClass, predClass)))

    # 2. Confidence loss
    # Bear in mind, for the boxes with no objects, we account for the confidence loss as well. 
    # To penalize the network for high confidence scores of the cells containing no objects. The 
    # cells that have no objects, have a confidence score of 0 in the target ground truth matrix.
    # Thus, the loss is calculated as follows: SUM_All_Cells_No_OBJ((C1-0)^2 + (C2-0)^2)
    confidenceLossObj = tf.math.reduce_sum(tf.math.square(maskObj * tf.subtract(targetConf, tf.expand_dims(respBox[...,0],-1))))
    confidenceLossNoObj =  lambdaNoObj * tf.reduce_sum(maskNoObj * tf.reduce_sum(tf.square(predConf), axis = -1, keepdims = True))
    
    # 3. Localization loss
    # Bear in mind that respBox is of the shape (...,5) and targetCoords dimension is (...,4) 
    xyLoss = (tf.reduce_sum(tf.square(tf.subtract(respBox[...,1:3], targetCoords[...,0:2])),-1,True))
    whLoss = (tf.reduce_sum(tf.square(tf.subtract(customSQRT2(respBox[...,1:3]), customSQRT2(targetCoords[...,0:2]))),-1,True))
    localizationLoss = lambdaCoord * (xyLoss + whLoss) 

    # Sum all the tree types of the errors
    return classificationLoss + confidenceLossNoObj + confidenceLossObj + localizationLoss

In [3]:
# YOLOv1 structure
YOLOv1_inputShape = (448,448,3) # Shape of the input image 
classNo = 1 # Number of classes we are trying to detect
input = tf.keras.layers.Input(shape=YOLOv1_inputShape)
leakyReLu = tf.keras.layers.LeakyReLU(negative_slope = .1)


# The backbone, Acts ads a feature extractor
# L1
x = tf.keras.layers.Conv2D(filters = 64, kernel_size=7, strides = 2, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(input)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L2
x = tf.keras.layers.Conv2D(filters = 192, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L3
x = tf.keras.layers.Conv2D(filters = 128, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 256, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 256, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L4
for _ in range(4):
    x = tf.keras.layers.Conv2D(filters = 256, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
    x = tf.keras.layers.Conv2D(filters = 512, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L5
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 2, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)

# L6
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)

# Neck
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4096)(x)
x = tf.keras.layers.Dense(7*7*(5*2+classNo), activation="sigmoid")(x)
x = tf.keras.layers.Dropout(.5)(x) # Dropout layer for avoiding overfitting
x = YOLOv1_LastLayer_Reshape((7,7,5*2+classNo))(x)
model = tf.keras.Model(inputs = input, outputs = x, name = "YOLOv1")

model.compile(loss = YOLOv1_loss ,optimizer = 'adam')

In [4]:
# See if the directory to save the checkpoints exists
if not os.path.isdir(f"{os.getcwd()}/model_data"):
    os.mkdir(f"{os.getcwd()}/model_data")

# Instantiate the checkpoint object
chkPoint = ModelCheckpoint(filepath='./model_data/model_{epoch:02d}-{val_loss:.2f}.keras',
                                    save_best_only=True,
                                    monitor='val_loss',
                                    mode='min',
                                    verbose=1
                            )

batch_size = 1
LR_schedule = [
    (0, 0.01),
    (75, 0.001),
    (105, 0.0001),
]

dfTrain = annotationsToDataframe(f"../data/labels/train", "txt")
trainingBatchGenerator = dataGenerator_YOLOv1(f"../data/images/train", batch_size, (448,448), dfTrain, 1, True)

dfTest = annotationsToDataframe(f"../data/labels/test", "txt")
testingBatchGenerator = dataGenerator_YOLOv1(f"../data/images/test", batch_size, (448,448), dfTrain, 1, True)

print(f"Training data: {trainingBatchGenerator.indexes.shape[0]}, testing data: {testingBatchGenerator.indexes.shape[0]}")

model.fit(x=trainingBatchGenerator,
        steps_per_epoch = int(trainingBatchGenerator.indexes.shape[0] // batch_size),
        epochs = 135,
        verbose = 1,
        validation_data = testingBatchGenerator,
        validation_steps = int(testingBatchGenerator.indexes.shape[0] // batch_size),
        callbacks = [
            customLearningRate(lrScheduler, LR_schedule),
            chkPoint,
        ]
)

Training data: 982, testing data: 983
Updated the learning rate at epoch NO. 0. New learning rate: 0.01
Epoch 1/135


I0000 00:00:1722355221.204585   14279 service.cc:146] XLA service 0x7960e40272b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722355221.204636   14279 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-07-30 19:30:21.435440: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-30 19:30:22.005874: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-07-30 19:30:23.963267: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-30 19:30:23.963319: W external/local_tsl/tsl/framework/bfc_allocator.cc:291]

  1/982 ━━━━━━━━━━━━━━━━━━━━ 4:24:49 16s/step - loss: 18.7119

2024-07-30 19:30:31.309392: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_12', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1722355231.342839   14279 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 77/982 ━━━━━━━━━━━━━━━━━━━━ 1:24 93ms/step - loss: 15.9662